<a href="https://colab.research.google.com/github/barauna-lo/Code_Library/blob/main/PSDgfec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy import stats, optimize
import numpy as np
import math


def psd(data):
	"""Calcula o PSD de uma série temporal."""
	
	# Define um intervalo para realizar o ajuste da reta
	INICIO = 10
	FIM = 1000
	# O vetor com o tempo é o tamanho do número de pontos
	N = len(data)
	tempo = np.arange(len(data))
	# Define a frequência de amostragem
	dt = (tempo[-1] - tempo[0] / (N - 1))
	fs = 1 / dt
	# Calcula o PSD utilizando o MLAB
	power, freqs = mlab.psd(data, Fs = fs, NFFT = N, scale_by_freq = False)
	# Calcula a porcentagem de pontos utilizados na reta de ajuste
	totalFrequencias = len(freqs)
	totalPSD = FIM - INICIO
	porcentagemPSD = int(100 * totalPSD / totalFrequencias)
	# Seleciona os dados dentro do intervalo de seleção
	xdata = freqs[INICIO:FIM]
	ydata = power[INICIO:FIM]
	# Simula o erro
	yerr = 0.2 * ydata
	# Define uma função para calcular a Lei de Potência
	powerlaw = lambda x, amp, index: amp * (x**index)
	# Converte os dados para o formato LOG
	logx = np.log10(xdata)
	logy = np.log10(ydata)
	# Define a função para realizar o ajuste
	fitfunc = lambda p, x: p[0] + p[1] * x
	errfunc = lambda p, x, y, err: (y - fitfunc(p, x)) / err    
	logyerr = yerr / ydata
	# Calcula a reta de ajuste
	pinit = [1.0, -1.0]
	out = optimize.leastsq(errfunc, pinit, args = (logx, logy, logyerr), full_output = 1)    
	pfinal = out[0]
	covar = out[1]
	index = pfinal[1]
	amp = 10.0 ** pfinal[0]
	indexErr = np.sqrt(covar[0][0])
	ampErr = np.sqrt(covar[1][1]) * amp
	# Retorna os valores obtidos
	return pfinal[1]